In [1]:
# Install required libraries (run this cell if not already installed)
!pip install earthengine-api geemap ipywidgets
import ee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00


In [2]:
cloud_project = 'urban-heat-island-454904'

try:
  ee.Initialize(project=cloud_project)
except:
  ee.Authenticate()
  ee.Initialize(project=cloud_project)

In [3]:
import matplotlib.colors as mcolors
import numpy as np
import ipywidgets as widgets
import geemap
import matplotlib.pyplot as plt

In [4]:
# Define Pune geometry (20 km buffer to match your LST dataset)
pune_point = ee.Geometry.Point([73.8567, 18.5204])
pune_area = pune_point.buffer(20000).bounds()

# Custom color palette for plotting
colors = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8',
          '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
cmap = mcolors.LinearSegmentedColormap.from_list('custom', colors, N=51)
norm = mcolors.BoundaryNorm(np.arange(0, 52, 1), ncolors=51)

def fetch_lst_image(year, month, return_kelvin=False):
    """
    Fetch the monthly mean MODIS LST image for Pune.

    Parameters:
      - year (int): Year of interest.
      - month (int): Month of interest.
      - return_kelvin (bool): If True, return raw LST in Kelvin;
                              otherwise convert to Celsius.

    Returns:
      - ee.Image: The monthly averaged LST image clipped to the Pune area.
    """
    # Define start and end dates for the month
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month+1:02d}-01' if month < 12 else f'{year+1}-01-01'

    dataset = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(start_date, end_date) \
        .filterBounds(pune_area)

    if return_kelvin:
        # Process the LST in Kelvin
        def convert_to_lst(img):
            return img.select('LST_Day_1km') \
                      .multiply(0.02) \
                      .rename('LST_K') \
                      .copyProperties(img, ['system:time_start'])
    else:
        # Convert the LST from Kelvin to Celsius
        def convert_to_lst(img):
            return img.select('LST_Day_1km') \
                      .multiply(0.02) \
                      .subtract(273.15) \
                      .rename('LST_C') \
                      .copyProperties(img, ['system:time_start'])

    lst_image = dataset.map(convert_to_lst).mean().clip(pune_area)
    return lst_image

# Global dictionary to store LST data as NumPy arrays.
lst_dataset = {}

def process_lst_data(year, month, use_kelvin=False):
    """
    Fetch the LST image from Earth Engine, convert it to a NumPy array,
    store it in a global variable, and plot the image.

    Parameters:
      - year (int): Year.
      - month (int): Month.
      - use_kelvin (bool): If True, process the image in Kelvin.
    """
    print(f"Processing LST image for {month:02d}/{year} with use_kelvin={use_kelvin}...")
    # Fetch the Earth Engine image.
    ee_img = fetch_lst_image(year, month, return_kelvin=use_kelvin)

    # Convert the image to a NumPy array.
    arr = geemap.ee_to_numpy(ee_img, region=pune_area, scale=1000)
    arr = np.squeeze(arr)

    # Save to global variable using a key that includes the conversion mode.
    key = f"{year}_{month:02d}_{'K' if use_kelvin else 'C'}"
    lst_dataset[key] = arr
    print(f"Data stored in variable 'lst_dataset' under key '{key}'.")

    # Plotting the image.
    plt.figure(figsize=(10, 8))
    if use_kelvin:
        # Adjust normalization for Kelvin (example range)
        kelvin_norm = mcolors.Normalize(vmin=290, vmax=320)
        img = plt.imshow(arr, cmap=cmap, norm=kelvin_norm)
        plt.title(f"LST Image (K) - {month:02d}/{year}")
    else:
        img = plt.imshow(arr, cmap=cmap, norm=norm)
        plt.title(f"LST Image (°C) - {month:02d}/{year}")
    plt.axis('off')
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Temperature')
    plt.show()

# Create interactive widgets for year, month, and Kelvin toggle.
year_widget = widgets.IntSlider(value=2010, min=2003, max=2018, step=1, description='Year:')
month_widget = widgets.Dropdown(
    options=[(name, i+1) for i, name in enumerate([
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ])],
    value=1,
    description='Month:'
)
kelvin_toggle = widgets.ToggleButton(
    value=False,
    description='Use Kelvin',
    tooltip='Toggle to process image in Kelvin'
)

# Set up the interactive UI.
ui = widgets.VBox([widgets.HBox([year_widget, month_widget]), kelvin_toggle])
out = widgets.interactive_output(process_lst_data, {'year': year_widget, 'month': month_widget, 'use_kelvin': kelvin_toggle})
display(ui, out)

print("The selected LST image is visualized and stored in the variable 'lst_dataset'.")

Output()

The selected LST image is visualized and stored in the variable 'lst_dataset'.


In [5]:
# Define Pune geometry (20 km buffer to match LST)
pune_point = ee.Geometry.Point([73.8567, 18.5204])
pune_area = pune_point.buffer(20000).bounds()

# Load the annual landcover dataset and create a mosaic.
annual = ee.ImageCollection("projects/sat-io/open-datasets/GLC-FCS30D/annual")
annual_mosaic = annual.mosaic()

# Function to recode class values into sequential values starting from 1
def recode_classes(image):
    classes = [10, 11, 12, 20, 51, 52, 61, 62, 71, 72, 81, 82, 91, 92, 120, 121, 122,
               130, 140, 150, 152, 153, 181, 182, 183, 184, 185, 186, 187, 190, 200,
               201, 202, 210, 220, 0]
    new_values = list(range(1, len(classes) + 1))
    return image.remap(classes, new_values)

# Define a visualization palette (36 colors for 36 classes).
palette = [
  "#ffff64", "#ffff64", "#ffff00", "#aaf0f0", "#4c7300", "#006400", "#a8c800", "#00a000",
  "#005000", "#003c00", "#286400", "#285000", "#a0b432", "#788200", "#966400", "#964b00",
  "#966400", "#ffb432", "#ffdcd2", "#ffebaf", "#ffd278", "#ffebaf", "#00a884", "#73ffdf",
  "#9ebb3b", "#828282", "#f57ab6", "#66cdab", "#444f89", "#c31400", "#fff5d7", "#dcdcdc",
  "#fff5d7", "#0046c8", "#ffffff", "#ffffff"
]
landcover_cmap = mcolors.ListedColormap(palette)
landcover_norm = mcolors.BoundaryNorm(np.arange(1, 38), ncolors=36)

# Global dictionary to store processed landcover data (annual cadence)
landcover_dataset = {}

def process_landcover_data(year):
    """
    Fetch, process, and display the landcover image for the given year.
    For this dataset, band index = year - 1999 (e.g., for year 2003, band 'b4' is used).
    The processed NumPy array is stored in the global dictionary.
    """
    print(f"Processing landcover data for {year}...")
    # If data is already fetched, simply display it.
    if year in landcover_dataset:
        arr = landcover_dataset[year]
        print(f"Data for {year} loaded from variable.")
    else:
        # Compute the band index (band for year 2000 is 'b1')
        band_index = year - 1999
        band_name = 'b' + str(band_index)
        # Select the corresponding band and clip to Pune area.
        image = annual_mosaic.select(band_name).clip(pune_area)
        # Recode the landcover classes to sequential values.
        recoded = recode_classes(image)
        # Convert the Earth Engine image to a NumPy array at 30 m resolution.
        arr = geemap.ee_to_numpy(recoded, region=pune_area, scale=30)
        arr = np.squeeze(arr)
        # Store in the global dictionary.
        landcover_dataset[year] = arr
        print(f"Data for {year} stored in variable.")

    # Plot the landcover image.
    plt.figure(figsize=(8,8))
    img = plt.imshow(arr, cmap=landcover_cmap, norm=landcover_norm)
    plt.title(f"Landcover for {year}")
    plt.axis('off')
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Class Index')
    plt.show()

# Create an interactive slider (using the same style as for LST)
year_slider = widgets.IntSlider(value=2003, min=2003, max=2018, step=1, description='Year:')

# Set up the interactive output widget.
interactive_landcover = widgets.interactive_output(process_landcover_data, {'year': year_slider})

# Display the slider and the output.
display(year_slider, interactive_landcover)

print("Use the slider to view the annual landcover dataset. The data is stored in 'landcover_dataset'.")

IntSlider(value=2003, description='Year:', max=2018, min=2003)

Output()

Use the slider to view the annual landcover dataset. The data is stored in 'landcover_dataset'.


In [6]:
# Define Pune geometry (using a 20 km buffer)
pune_point = ee.Geometry.Point([73.8567, 18.5204])
pune_area = pune_point.buffer(20000).bounds()

# Global dictionary to store PM2.5 data as NumPy arrays (monthly cadence)
pm25_dataset = {}

def fetch_pm25_image(year, month):
    """
    Fetch the monthly mean PM2.5 image for Pune from the GLOBAL-SATELLITE-PM25/MONTHLY dataset.

    Parameters:
      - year (int): Year of interest.
      - month (int): Month of interest.

    Returns:
      - ee.Image: The monthly mean composite PM2.5 image clipped to the Pune area.
    """
    # Define start and end dates for the month.
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month+1:02d}-01" if month < 12 else f"{year+1}-01-01"

    # Load the PM2.5 image collection from the correct dataset.
    pm25_collection = ee.ImageCollection("projects/sat-io/open-datasets/GLOBAL-SATELLITE-PM25/MONTHLY") \
                        .filterDate(start_date, end_date) \
                        .filterBounds(pune_area)

    # Compute the monthly mean composite.
    pm25_image = pm25_collection.mean().clip(pune_area)
    return pm25_image

def plot_pm25_image(pm25_image, title="PM2.5 Image", scale=1000):
    """
    Convert the Earth Engine PM2.5 image to a NumPy array and plot it.
    """
    try:
        # Convert the image to a NumPy array.
        arr = geemap.ee_to_numpy(pm25_image, region=pune_area, scale=scale)
        arr = np.squeeze(arr)

        plt.figure(figsize=(10,8))
        # Display the image using the 'viridis' colormap (adjust if needed)
        img = plt.imshow(arr, cmap='viridis')
        plt.title(title)
        plt.axis('off')
        cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
        cbar.set_label('PM2.5 (µg/m³)')
        plt.show()
    except Exception as e:
        print("Error plotting PM2.5 image:", e)

def process_pm25_data(year, month):
    """
    Fetch the PM2.5 image for the specified month and year,
    convert it to a NumPy array, store it in the global variable,
    and display the image.
    """
    print(f"Processing PM2.5 data for {month:02d}/{year}...")
    # Fetch the Earth Engine image.
    ee_img = fetch_pm25_image(year, month)

    # Convert the image to a NumPy array.
    arr = geemap.ee_to_numpy(ee_img, region=pune_area, scale=1000)
    arr = np.squeeze(arr)

    # Store in the global dictionary using a key "year_month"
    key = f"{year}_{month:02d}"
    pm25_dataset[key] = arr
    print(f"Data stored in pm25_dataset under key '{key}'.")

    # Plot the PM2.5 image.
    plot_pm25_image(ee_img, title=f"PM2.5 - {month:02d}/{year}")

# Create interactive widgets for year and month.
year_widget_pm25 = widgets.IntSlider(value=2010, min=2003, max=2018, step=1, description='Year:')
month_widget_pm25 = widgets.Dropdown(
    options=[(name, i+1) for i, name in enumerate([
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ])],
    value=1,
    description='Month:'
)

# Set up the interactive UI.
ui_pm25 = widgets.HBox([year_widget_pm25, month_widget_pm25])
out_pm25 = widgets.interactive_output(process_pm25_data, {'year': year_widget_pm25, 'month': month_widget_pm25})
display(ui_pm25, out_pm25)

print("Use the widgets to fetch and visualize the PM2.5 dataset. Data is stored in 'pm25_dataset'.")


Output()

Use the widgets to fetch and visualize the PM2.5 dataset. Data is stored in 'pm25_dataset'.


In [7]:
# Define Pune geometry (20 km buffer)
pune_point = ee.Geometry.Point([73.8567, 18.5204])
pune_area = pune_point.buffer(20000).bounds()

# Dictionary mapping UHII collection names to their asset IDs.
uhii_collections = {
    "AMOD2": "projects/sat-io/open-datasets/UHII/AMOD2",
    "MOD1":  "projects/sat-io/open-datasets/UHII/MOD1",
    "MOD2":  "projects/sat-io/open-datasets/UHII/MOD2",
    "MYD1":  "projects/sat-io/open-datasets/UHII/MYD1",
    "MYD2":  "projects/sat-io/open-datasets/UHII/MYD2",
    "SAT":   "projects/sat-io/open-datasets/UHII/SAT",
    "SMOD2": "projects/sat-io/open-datasets/UHII/SMOD2",
    "SMYD1": "projects/sat-io/open-datasets/UHII/SMYD1"
}

# Global dictionary to store UHII monthly data.
# Keys will be strings like "MOD1_2010_03" for MOD1 data in March 2010.
uhii_monthly_dataset = {}

# Visualization parameters
vis_params = {
    'min': 1,
    'max': 25,
    'palette': ['#000004', '#1f0c48', '#550f6d', '#88226a',
                '#b63655', '#de4968', '#f87d46', '#fdca26', '#f0f921']
}

def process_uhii_monthly_data(collection_name, year, month):
    """
    For a given UHII collection, year, and month, filter the ImageCollection
    using calendar-based filters, take the first image (masking out non-positive values),
    clip it to Pune, convert it to a NumPy array at 300 m resolution,
    store it in a global dictionary, and display the image.

    Parameters:
      - collection_name (str): One of the keys in uhii_collections.
      - year (int): The year (2003-2018).
      - month (int): The month (1-12).
    """
    print(f"Processing {collection_name} for {year}-{month:02d}...")

    # Load the collection using its asset ID.
    asset_id = uhii_collections[collection_name]
    col = ee.ImageCollection(asset_id)

    # Filter by year and month using calendarRange filters.
    filtered = col.filter(ee.Filter.calendarRange(year, year, 'year')) \
                  .filter(ee.Filter.calendarRange(month, month, 'month')) \
                  .filterBounds(pune_area)

    count = filtered.size().getInfo()
    if count == 0:
        print(f"No image available for {collection_name} in {year}-{month:02d}.")
        return

    # Get the first image and apply a mask: keep pixels where the value > 0.
    uhi = ee.Image(filtered.first())
    uhi = uhi.mask(uhi.gt(0)).clip(pune_area)

    # Convert the image to a NumPy array at 300 m resolution.
    arr = geemap.ee_to_numpy(uhi, region=pune_area, scale=300)
    arr = np.squeeze(arr)

    # Create a key for storage.
    key = f"{collection_name}_{year}_{month:02d}"
    uhii_monthly_dataset[key] = arr
    print(f"Data stored under key '{key}'.")

    # Plot the image.
    plt.figure(figsize=(10,8))
    img = plt.imshow(arr, cmap='inferno', vmin=vis_params['min'], vmax=vis_params['max'])
    plt.title(f"UHII {collection_name} - {year}-{month:02d}")
    plt.axis("off")
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label("UHII Intensity")
    plt.show()

# Create interactive widgets.
collection_widget = widgets.Dropdown(
    options=list(uhii_collections.keys()),
    value="AMOD2",
    description="Collection:"
)
year_widget = widgets.IntSlider(value=2010, min=2003, max=2018, step=1, description="Year:")
month_widget = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(
        ['January','February','March','April','May','June',
         'July','August','September','October','November','December'], start=1)],
    value=1,
    description="Month:"
)

# Set up interactive output.
ui = widgets.VBox([widgets.HBox([collection_widget, year_widget, month_widget])])
out = widgets.interactive_output(process_uhii_monthly_data,
                                 {'collection_name': collection_widget,
                                  'year': year_widget,
                                  'month': month_widget})
display(ui, out)

print("Use the widgets to fetch and visualize UHII monthly data from 2003 to 2018. Data is stored in 'uhii_monthly_dataset'.")

Output()

Use the widgets to fetch and visualize UHII monthly data from 2003 to 2018. Data is stored in 'uhii_monthly_dataset'.


In [15]:
# Define Pune geometry (20 km buffer)
pune_point = ee.Geometry.Point([73.8567, 18.5204])
pune_area = pune_point.buffer(20000).bounds()

# Dictionary mapping UHII collection names to their asset IDs.
uhii_collections = {
    "AMOD2": "projects/sat-io/open-datasets/UHII/AMOD2",
    "MOD1":  "projects/sat-io/open-datasets/UHII/MOD1",
    "MOD2":  "projects/sat-io/open-datasets/UHII/MOD2",
    "MYD1":  "projects/sat-io/open-datasets/UHII/MYD1",
    "MYD2":  "projects/sat-io/open-datasets/UHII/MYD2",
    "SAT":   "projects/sat-io/open-datasets/UHII/SAT",
    "SMOD2": "projects/sat-io/open-datasets/UHII/SMOD2",
    "SMYD1": "projects/sat-io/open-datasets/UHII/SMYD1"
}

# Visualization parameters (adjust if necessary).
vis_params = {
    'min': 1,
    'max': 25,
    'palette': ['#000004', '#1f0c48', '#550f6d', '#88226a',
                '#b63655', '#de4968', '#f87d46', '#fdca26', '#f0f921']
}

def process_combined_uhii_monthly_data(year, month):
    """
    For a given year and month, this function:
      1. Iterates through all UHII collections,
      2. Filters each collection by the specified year, month, and the Pune area,
      3. If there are images, computes the monthly mean image,
         masks out non-positive pixels, and clips to Pune,
      4. Combines all the available collection images into a multi-band image,
      5. Computes the pixel-wise average across all bands,
      6. Converts the result to a NumPy array (at 300 m resolution),
      7. Plots the average image.
    """
    band_images = []

    # Process each UHII collection.
    for key, asset_id in uhii_collections.items():
        collection = ee.ImageCollection(asset_id) \
                        .filter(ee.Filter.calendarRange(year, year, 'year')) \
                        .filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filterBounds(pune_area)

        count = collection.size().getInfo()
        if count > 0:
            # Compute the monthly mean image and rename its band.
            image = collection.mean().rename(key)
            # Mask out pixels with non-positive values and clip to the Pune area.
            image = image.updateMask(image.gt(0)).clip(pune_area)
            band_images.append(image)
        else:
            print(f"No data available for {key} in {year}-{month:02d}.")

    if len(band_images) == 0:
        print(f"No UHII data available for {year}-{month:02d}.")
        return

    # Combine the images from all collections into a multi-band image.
    combined = ee.Image.cat(band_images)

    # Compute the pixel-wise average across all bands.
    avg_image = combined.reduce(ee.Reducer.mean()).rename('avg_UHII')

    # (Optional: apply scaling if needed, e.g., multiply by a factor like 0.01)
    # avg_image = avg_image.multiply(0.01)

    # Convert the Earth Engine image to a NumPy array at 300 m resolution.
    arr = geemap.ee_to_numpy(avg_image, region=pune_area, scale=300)
    arr = np.squeeze(arr)

    # Plot the averaged image.
    plt.figure(figsize=(10, 8))
    img_plot = plt.imshow(arr, cmap='inferno', vmin=vis_params['min'], vmax=vis_params['max'])
    plt.title(f"Combined Average UHII - {year}-{month:02d} for Pune")
    plt.axis("off")
    cbar = plt.colorbar(img_plot, fraction=0.046, pad=0.04)
    cbar.set_label("UHII Intensity")
    plt.show()

# Create interactive widgets for selecting year and month.
year_widget = widgets.IntSlider(value=2010, min=2003, max=2018, step=1, description="Year:")
month_widget = widgets.Dropdown(options=[("January", 1), ("February", 2), ("March", 3),
                                           ("April", 4), ("May", 5), ("June", 6),
                                           ("July", 7), ("August", 8), ("September", 9),
                                           ("October", 10), ("November", 11), ("December", 12)],
                                 value=1, description="Month:")

# Link the interactive widgets to the processing function.
ui = widgets.HBox([year_widget, month_widget])
out = widgets.interactive_output(process_combined_uhii_monthly_data, {'year': year_widget, 'month': month_widget})
display(ui, out)

print("Use the widgets to browse and visualize the combined average UHII data for Pune City.")

Output()

Use the widgets to browse and visualize the combined average UHII data for Pune City.
